## 4. Add statistics to segments shapefile

Use this notebook to add some hydrologic statistic attributes to the segments shapefile. These stats are a subset of the full dataset designed for quick display on the webapp via WMS / WFS. Unless otherwise noted, we are providing the difference between static landcover, Maurer model, historical scenario, 1976-2005 era and static landcover, multi-model ensemble, RCP 8.5 scenario, 2071-2100 era.

### Which stats?

#### Magnitude

- `ma99_hist`: Mean Annual Flow (static landcover, Maurer model, historical scenario, 1976-2005 era)
  - **note: this is for the default map view on the web app**
  - statistic units = cfs
  - difference units = NA

- `ma99_diff`: Mean Annual Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma12_diff`: Mean January Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma13_diff`: Mean February Flow
  - difference between mean of ensemble values and Maurer historical 
  - statistic units = cfs
  - difference units = percentage +/-

- `ma14_diff`: Mean March Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma15_diff`: Mean April Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma16_diff`: Mean May Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma17_diff`: Mean June Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma18_diff`: Mean July Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma19_diff`: Mean August Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma20_diff`: Mean September Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma21_diff`: Mean October Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma22_diff`: Mean November Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma23_diff`: Mean December Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

#### Duration

- `dh1_diff`: Annual Maximum Daily Flow
  - difference between maximum of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `dl1_diff`: Annual Minimum Daily Flow
  - difference between minimum of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `dh15_diff`: High Flow Pulse Duration
  - difference between maximum of ensemble values and Maurer historical
  - statistic units = days per year
  - difference units = percentage +/-

- `dl16_diff`: Low Flow Pulse Duration
  - difference between maximum of ensemble values and Maurer historical
  - statistic units = days per year
  - difference units = percentage +/-

#### Frequency

- `fh1_diff`: High Flood Pulse Count
  - difference between maximum of ensemble values and Maurer historical
  - statistic units = events per year
  - difference units = percentage +/-

- `fl1_diff`: Low Flood Pulse Count
  - difference between maximum of ensemble values and Maurer historical
  - statistic units = events per year
  - difference units = percentage +/-

#### Timing

- `th1_diff`: Julian Date of Annual Maximum
  - difference between mean of ensemble values and Maurer historical
  - statistic units = day of year
  - difference units = absolute +/-

- `tl1_diff`: Julian Date of Annual Minimum
  - difference between mean of ensemble values and Maurer historical
  - statistic units = day of year
  - difference units = absolute +/-



### Setup

In [ ]:
import xarray as xr

Load combined stats file and subset variables.

In [ ]:

vars = {"ma99":{
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma12":{
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        },
        "ma13":{
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma14":{
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma15":{
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma16":{
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        },
        "ma17": {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma18": {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma19": {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma20": {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma21": {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma22": {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        },
        "ma23": {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        },
        "dh1": {
                "aggregation":"maximum",
                "units":"cfs",
                "difference":"percentage"
        },
        "dl1": {
                "aggregation":"minimum",
                "units":"cfs",
                "difference":"percentage"
        },
        "dh15": {
                "aggregation":"maximum",
                "units":"days per year",
                "difference":"percentage"
        },
        "dh16": {
                "aggregation":"minimum",
                "units":"days per year",
                "difference":"percentage"
        },
        "fh1": {
                "aggregation":"maximum",
                "units":"events per year",
                "difference":"percentage"
        },
        "fl1": {
                "aggregation":"minimum",
                "units":"events per year",
                "difference":"percentage"
        },
        "th1": {
                "aggregation":"maximum",
                "units":"day of year",
                "difference":"absolute"
        },
        "tl1": {
                "aggregation":"minimum",
                "units":"day of year",
                "difference":"absolute"
        },
}

ds = xr.open_dataset("/import/beegfs/CMIP6/jdpaul3/hydroviz_data/maurer/nc_stats_fix/seg.nc")
ds = ds[list(vars.keys())]

Decode the dataset dimensions using the encoding attributes.

In [ ]:
# decode routine

Compute a model ensemble for the static + RCP 8.5 + late century era and reference in a new dataset. Extract the Maurer historical data to a new dataset as well.

In [ ]:
ds_proj = ds.sel(landcover="static", scenario="rcp85", era="2071-2100")
# take an average across the model dimension
ds_proj = ds_proj.mean(dim="model")



In [ ]:

ds_hist = ds.sel(landcover="static", model="Maurer", scenario="historical", era="1971-2000")